# Исследование надежности заемщиков

<h2> Введение <a class="tocSkip"> </h2>
    
Целью данной работы: Исследование корреляции семейного положения, количества детей клиента и факта погашения кредита в срок. 
    
Входными данные:  Статистика о платёжеспособности клиентов от банка.
    
Работа включает следующие этапы:
    
* Удаление пропусков в данных
* Обработка аномальных значений в данных
* Обработка дубликатов
* Категоризация данных

## Изучаем данные

In [1]:
# Импорты:
import pandas as pd
import numpy as np

# Читаем csv-файл с помощью pandas:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
display(data.head(5));

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
# Вывожу количество пропущенных значений для каждого столбца.
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
print(f"Количество пропусков:\n - days_employed: {data['days_employed'].isna().sum()}\n - total_income: {data['total_income'].isna().sum()}")

Количество пропусков:
 - days_employed: 2174
 - total_income: 2174


In [6]:
# На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски 
# в этом поле медианным значением по каждому типу из столбца income_type.
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [7]:
# Делаем отрицательные значения в поле 'days_employed' положительными
data['days_employed'] = data['days_employed'].abs()

In [8]:
# Для каждого типа занятости выведем медианное значение трудового стажа days_employed в днях.
round(data.groupby('income_type')['days_employed'].agg('median'), 2)

income_type
безработный        366413.65
в декрете            3296.76
госслужащий          2689.37
компаньон            1547.38
пенсионер          365213.31
предприниматель       520.85
сотрудник            1574.20
студент               578.75
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

In [9]:
# Выведем перечень уникальных значений столбца children.
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [10]:
# В столбце children есть два аномальных значения. Удалим строки, 
# в которых встречаются такие аномальные значения из датафрейма data.
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
# Ещё раз выведем перечень уникальных значений столбца children, чтобы убедиться, что артефакты удалены.
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [12]:
# Заполним пропуски в столбце days_employed медианными значениями по каждого типа занятости income_type.
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
# Убедимся, что все пропуски заполнены. 
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски заполнены.

### Изменение типов данных

In [14]:
# Заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype().
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [15]:
# Обработаем неявные дубликаты в столбце education. В этом столбце есть одни и те же значения, 
# но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру.
data['education'] = data['education'].str.lower()

In [16]:
# Выведем на экран количество строк-дубликатов в данных.
data.duplicated().sum()

71

In [17]:
# Удалим полные дубликаты.
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создаем в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 назначаем категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [20]:
# Выведем на экран перечень уникальных целей взятия кредита из столбца purpose.
display(data['purpose'].unique());

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:**

- 'операции с автомобилем'
- 'операции с недвижимостью'
- 'проведение свадьбы'
- 'получение образования'

**Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.**

**Используем собственную функцию с именем categorize_purpose() и метод apply(). Изучим данные в столбце purpose и определим, какие подстроки помогут вам правильно определить категорию.**

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуем данные и ответим на вопросы бизнеса

In [23]:
# Посмотрим на обновлённую таблицу
display(data.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [24]:
pd.pivot_table(data, index=['children'], values=['debt'], aggfunc={'debt':['count', 'sum', 'mean']})

debt                
          count      mean   sum
children                       
0         14091  0.075438  1063
1          4808  0.092346   444
2          2052  0.094542   194
3           330  0.081818    27
4            41  0.097561     4
5             9  0.000000     0

In [25]:
data_temp_ch = data.groupby('children')['debt'].agg(['count', 'sum', 'mean']).sort_values(by='mean')
 
# Воспользуемся f строкой # переводим данные в столбце mean в проценты.
data_temp_ch['mean'] = [f'{x:,.2%}' for x in data_temp_ch['mean']] 

data_temp_ch.columns = ['Количество заёмщиков', 'Количество должников', 'Средний процент просрочки']

# Смотрим на табличку.
data_temp_ch

,Количество заёмщиков,Количество должников,Средний процент просрочки
children,,,
5,9,0,0.00%
0,14091,1063,7.54%
3,330,27,8.18%
1,4808,444,9.23%
2,2052,194,9.45%
4,41,4,9.76%


**Вывод:**

* По полученным данным можно сделать вывод о том, что самая надёжная категория - заёмщики, которые не имеют детей. 

* Согласно приведённой выше таблице, мы видим, что самой ненадёжной категорией являются заёмщики, имеющие четверых детей. В категории заёмщиков, имеющих пятерых детей, мы видим нулевой процент просрочки, но данная категория слишком малочисленная, относительно других категорий, для того, чтобы признать её самой надёжной. 

* К тому же для всех категорий, с меньшим количеством детей и большим количеством данных по этим категориям мы видим чёткую зависимость - чем больще у заёмщиков детей, тем выше процент просрочки по кредитам.

* Также следует отметить, что выборки не сбалансированны между собой, а значит по категориям с небольшим количеством данных мы не сможем сделать достоверных выводов.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
data_temp_fs = data.groupby('family_status_id')['debt'].agg(['count', 'sum', 'mean']).sort_values(by='mean')
 
# Воспользуемся f строкой # переводим данные в столбце mean в проценты.
data_temp_fs['mean'] = [f'{x:,.2%}' for x in data_temp_fs['mean']] 

data_temp_fs.columns = ['Количество заёмщиков', 'Количество должников', 'Средний процент просрочки']

# Смотрим на табличку.
data_temp_fs

,Количество заёмщиков,Количество должников,Средний процент просрочки
family_status_id,,,
2,951,63,6.62%
3,1189,84,7.06%
0,12261,927,7.56%
1,4134,385,9.31%
4,2796,273,9.76%


In [27]:
display(data['family_status'].unique())

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

**Описание категорий:**

- 0 категория - женат / замужем   
- 1 категория - гражданский брак
- 2 категория - вдовец / вдова   
- 3 категория - в разводе
- 4 категория - не женат / не замужем  

**Вывод:**

* Просматривается чёткая зависимость между семейным положением и возвратом кредита в срок.
* На диаграмме видно, что больше всего задолженностей по возврату кредитов имели люди не состоящие в браке. 
* Мешьше всего вдовцы и вдовы.
* Также следует отметить, что выборки не сбалансированны между собой, а значит по категориям с небольшим количеством данных мы не сможем сделать достоверных выводов.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
data_temp_ti = data.groupby('total_income_category')['debt'].agg(['count', 'sum', 'mean']).sort_values(by='mean')
 
# Воспользуемся f строкой # переводим данные в столбце mean в проценты
data_temp_ti['mean'] = [f'{x:,.2%}' for x in data_temp_ti['mean']] 

data_temp_ti.columns = ['Количество заёмщиков', 'Количество должников', 'Средний процент просрочки']

# Смотрим на табличку
data_temp_ti

,Количество заёмщиков,Количество должников,Средний процент просрочки
total_income_category,,,
D,349,21,6.02%
B,5014,354,7.06%
A,25,2,8.00%
C,15921,1353,8.50%
E,22,2,9.09%


**Описание категорий**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

**Вывод:**

* По полученным данным можно выявить чёткую зависимость между уровнем дохода заёмщиков и их возможностью вернуть кредит в срок. 
* Самыми ненадёжными являются заёмщики имеющие доход 0–30000 и относящиеся к категории 'E'. 
* Самыми надёжными являются категории с доходами 30001–50000 — 'D' и 200001–1000000 — 'B'.
* По категориям 1000001 и выше — 'A' и 0–30000 — 'E' данных крайне мало для того, чтобы дать поним однозначную оценку.
* Также следует отметить, что выборки не сбалансированны между собой, а значит по категориям с небольшим количеством данных мы не сможем сделать достоверных выводов.

### Как разные цели кредита влияют на его возврат в срок?

In [29]:
data_temp_pc = data.groupby('purpose_category')['debt'].agg(['count', 'sum', 'mean']).sort_values(by='mean')
 
# Воспользуемся f строкой # переводим данные в столбце mean в проценты
data_temp_pc['mean'] = [f'{x:,.2%}' for x in data_temp_pc['mean']] 

data_temp_pc.columns = ['Количество заёмщиков', 'Количество должников', 'Средний процент просрочки']

# Смотрим на табличку
data_temp_pc

,Количество заёмщиков,Количество должников,Средний процент просрочки
purpose_category,,,
операции с недвижимостью,10751,780,7.26%
проведение свадьбы,2313,183,7.91%
получение образования,3988,369,9.25%
операции с автомобилем,4279,400,9.35%


**Вывод:**

* Прослеживается зависимость между целью кредита и наличием у заёмщика задолженности по возврату. 
* Cамая надёжная категория заёмщиков - "операции с недвижимостью".
* Самая ненадёжная категория заёмщиков - "операции с автомобилем".
* Также следует отметить, что выборки не сбалансированны между собой, а значит по категориям с небольшим количеством данных мы не сможем сделать достоверных выводов.

### Возможные причины появления пропусков в исходных данных:

Пропуски наблюдаются в столбцах days_employed и total_income - их количество равно и присутствуют они в одних и тех же сроках. Так как пропуски имеют нерегулярный характер - это может быть человеческим фактором, но так как пропуски встречаются в одних и тех же строках, так же это может быть и по техническим причинам.

### Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Заполнять пропуски в количественных данных медианными значениями лучшее решение по причине того, что это не приведёт к искажению данных и соответственно не окажет негативного влияния на результаты исследования.

## Общий вывод

**В данном исследовании мы оценивали надёжность заёмщиков по 4-м категориям, таким как:**

* Наличие детей
* Семейный статус
* Уровень дохода
* Цель кредита

**На этапе предобработки данных мы столкнулись со следующими проблемами:**

* Наличие пропусков в данных. Так как пропуски имеют нерегулярный характер - это может быть человеческим фактором, но так как пропуски встречаются в одних и тех же строках, так же это может быть и по техническим причинам.
* Наличие неявных дубликатов в столбце 'education', обусловленных наличием различного регистра при обозначении категорий.
* Наличие в данных аномальных значений - отрицательное количество дней трудового стажа в столбце 'days_employed'. Так как аномальные значения имеют регулярный характер, их возникновение возможно по техническим причинам.
* Наличие аномальных значений в столбце 'children', вероятной причиной появления которых является человеческий фактор.
* Наличие явных дубликатов в перечне уникальных целей взятия кредита из столбца 'purpose', обусловленных человеческим фактором.

**По результатам проведённого исследования мы можем сделать следующие выводы:**

* Для категорий с небольшим количеством данных, относительно других категорий можно сделать вывод, что данные категории слабо заинтересованы в том, чтобы пользоваться кредитами - возможно эти категории недостаточно информированы.
* Так как выборки по различным категориям не сбалансированы между собой, по категориям с небольшим каличеством данных мы не сможем сделать достоверных выводов.

**Самыми надёжными категориями являются:**

* Заёмщики, которые не имеют детей
* Вдовцы и вдовы
* Заёмщики с доходом 30001–50000
* Заёмщики, производящие операции с недвижимостью

**Самыми ненадёжными категориями являются:**

* Заёмщики, имеющие четверых детей
* Не состоящие в браке
* Заёмщики имеющие доход 0–30000
* Заёмщики, производящие операции с автомобилем